In [4]:
# import splitfolders
# import os
# path = '/Users/namanh.ngco/Documents/FPT/AIL/emotion_detection/FER13_cleaner'
# print(os.listdir(path))

In [5]:
# splitfolders.ratio(path, output='data', ratio=(0.8, 0.1, 0.1))

In [6]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [7]:
tf.device("GPU")

In [8]:
# display, transform, read, split ...
import numpy as np
import cv2 as cv
import os
import splitfolders
import matplotlib.pyplot as plt

# tensorflow
import tensorflow.keras as keras
import tensorflow as tf

# image processing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

# model / neural network
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import regularizers

In [9]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import concatenate
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix


In [10]:
path = "./FER13/"

In [11]:
data_path = './negative/train/'
classes = os.listdir(data_path) #['happy', 'sad', 'fear', 'surprise', 'neutral', 'angry', 'disgust']
classes.remove('.DS_Store')
print(classes)
len_of_class = []
for folder in classes:
    if folder == '.DS_Store':
        continue
    len_of_class.append(len(os.listdir(data_path+folder)))
print(len_of_class)

['sad', 'fear', 'angry', 'disgust']
[4685, 2535, 2832, 2115]


In [12]:
datagen = ImageDataGenerator(rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [13]:
# Define data augmentation function
def data_augmentation(img_path: str, step: int, save_to_dir, save_format):
    # load img to array
    img = load_img(img_path)
    img_array = img_to_array(img)
    img_array = img_array.reshape((1,) + img_array.shape)

    # augmentation
    i = 0
    for batch in datagen.flow(img_array, batch_size=1, save_to_dir=save_to_dir, save_format=save_format):
        i += 1
        if i > step:
            break

In [14]:
def img_path_list(path: str):
    img_list = os.listdir(path)
    img_path_list = []
    for img in img_list:
        img_path_list.append(path + img)
    return img_path_list

In [15]:
# disgust_img_train_list = img_path_list(data_path + 'train/' + 'disgust/')
# print(disgust_img_train_list)

In [16]:
# disgust_img_val_list = img_path_list(data_path + 'val/' + 'disgust/')
# print(disgust_img_val_list)

In [17]:
# for file in disgust_img_train_list:
#     data_augmentation(file, 2, data_path + 'train/' + 'disgust/', 'jpg')
# print(len(os.listdir(data_path + 'train/' + 'disgust')))

In [18]:
# for file in disgust_img_val_list:
#     data_augmentation(file, 2, data_path + 'val/' + 'disgust/', 'jpg')
# print(len(os.listdir(data_path + 'val/' + 'disgust/')))

In [19]:
train_generator = datagen.flow_from_directory(
    directory = path + 'train/',
    classes = classes,
    batch_size = 32,
    target_size = (48, 48),
    class_mode = 'binary',
    color_mode = "grayscale",
)

Found 12167 images belonging to 4 classes.


In [20]:
test_generator = datagen.flow_from_directory(
    directory = path + 'test/',
    classes = classes,
    batch_size = 32,
    target_size = (48, 48),
    class_mode = 'binary',
    color_mode = "grayscale",

)

Found 3301 images belonging to 4 classes.


In [21]:
# valid_generator = datagen.flow_from_directory(
#     directory = path + 'val/',
#     classes = classes,
#     batch_size = 32,
#     target_size = (48, 48),
#     class_mode = 'binary',
#     color_mode = "grayscale",

# )

In [22]:
# img_check = image.load_img('/Users/namanh.ngco/Documents/FPT/AIL/emotion_detection/data/train/disgust/Training_81074213.jpg', '/Users/namanh.ngco/Documents/FPT/AIL/emotion_detection/data/train/disgust/Training_94620136.jpg')
# x = img_to_array(img_check)
# x.shape

In [23]:
# resnet_50 = ResNet50(include_top=False, weights='imagenet', input_shape=(48, 48, 3))
# for layer in resnet_50.layers:
#     layer.trainable = False

In [24]:
# # build the entire model
# x = resnet_50.output
# x = layers.GlobalAvgPool2D()(x)
# x = layers.Dense(512, activation = 'relu',kernel_regularizer=regularizers.l2(0.01))(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(256, activation = 'relu',kernel_regularizer=regularizers.l2(0.01))(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(128, activation = 'relu')(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(64, activation = 'relu')(x)
# x = layers.Dropout(0.5)(x)
# predictions = layers.Dense(3, activation='softmax')(x)
# model = Model(inputs = resnet_50.input, outputs = predictions)

In [25]:
# def FER_Model(input_shape=(48,48,1)):
#     # first input model
#     visible = Input(shape=input_shape, name='input')
#     num_classes = 3
#     #the 1-st block
#     conv1_1 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_1')(visible)
#     conv1_1 = BatchNormalization()(conv1_1)
#     conv1_2 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_2')(conv1_1)
#     conv1_2 = BatchNormalization()(conv1_2)
#     pool1_1 = MaxPooling2D(pool_size=(2,2), name = 'pool1_1')(conv1_2)
#     drop1_1 = Dropout(0.3, name = 'drop1_1')(pool1_1)#the 2-nd block
#     conv2_1 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_1')(drop1_1)
#     conv2_1 = BatchNormalization()(conv2_1)
#     conv2_2 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_2')(conv2_1)
#     conv2_2 = BatchNormalization()(conv2_2)
#     conv2_3 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_3')(conv2_2)
#     conv2_2 = BatchNormalization()(conv2_3)
#     pool2_1 = MaxPooling2D(pool_size=(2,2), name = 'pool2_1')(conv2_3)
#     drop2_1 = Dropout(0.3, name = 'drop2_1')(pool2_1)#the 3-rd block
#     conv3_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_1')(drop2_1)
#     conv3_1 = BatchNormalization()(conv3_1)
#     conv3_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_2')(conv3_1)
#     conv3_2 = BatchNormalization()(conv3_2)
#     conv3_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_3')(conv3_2)
#     conv3_3 = BatchNormalization()(conv3_3)
#     conv3_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_4')(conv3_3)
#     conv3_4 = BatchNormalization()(conv3_4)
#     pool3_1 = MaxPooling2D(pool_size=(2,2), name = 'pool3_1')(conv3_4)
#     drop3_1 = Dropout(0.3, name = 'drop3_1')(pool3_1)#the 4-th block
#     conv4_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_1')(drop3_1)
#     conv4_1 = BatchNormalization()(conv4_1)
#     conv4_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_2')(conv4_1)
#     conv4_2 = BatchNormalization()(conv4_2)
#     conv4_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_3')(conv4_2)
#     conv4_3 = BatchNormalization()(conv4_3)
#     conv4_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_4')(conv4_3)
#     conv4_4 = BatchNormalization()(conv4_4)
#     pool4_1 = MaxPooling2D(pool_size=(2,2), name = 'pool4_1')(conv4_4)
#     drop4_1 = Dropout(0.3, name = 'drop4_1')(pool4_1)

#     #the 5-th block
#     conv5_1 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_1')(drop4_1)
#     conv5_1 = BatchNormalization()(conv5_1)
#     conv5_2 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_2')(conv5_1)
#     conv5_2 = BatchNormalization()(conv5_2)
#     conv5_3 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_3')(conv5_2)
#     conv5_3 = BatchNormalization()(conv5_3)
#     conv5_4 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_4')(conv5_3)
#     conv5_3 = BatchNormalization()(conv5_3)
#     pool5_1 = MaxPooling2D(pool_size=(2,2), name = 'pool5_1')(conv5_4)
#     drop5_1 = Dropout(0.3, name = 'drop5_1')(pool5_1)#Flatten and output
#     flatten = Flatten(name = 'flatten')(drop5_1)
#     ouput = Dense(num_classes, activation='softmax', name = 'output')(flatten)# create model 
#     model = Model(inputs =visible, outputs = ouput)
#     # summary layers
#     print(model.summary())

#     return model

In [26]:
visible = Input(shape=(48,48,1), name='input')
num_classes = 4
conv2_1 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_1')(visible)
conv2_1 = BatchNormalization()(conv2_1)
conv2_2 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_2')(conv2_1)
conv2_2 = BatchNormalization()(conv2_2)
conv2_3 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_3')(conv2_2)
conv2_2 = BatchNormalization()(conv2_3)
pool2_1 = MaxPooling2D(pool_size=(2,2), name = 'pool2_1')(conv2_3)
drop2_1 = Dropout(0.3, name = 'drop2_1')(pool2_1)
pool3_1 = MaxPooling2D(pool_size=(2,2), name = 'pool3_1')(drop2_1)
drop3_1 = Dropout(0.3, name = 'drop3_1')(pool3_1)

flatten = Flatten(name = 'flatten')(drop3_1)
ouput = Dense(num_classes, activation='softmax', name = 'output')(flatten)# create model 
model = Model(inputs =visible, outputs = ouput)

In [27]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 48, 48, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2_1 (Conv2D)                │ (None, 48, 48, 128)    │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 48, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2_2 (Conv2D)                │ (None, 48, 48, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 48, 48, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2_3 (Conv2D)                │ (None, 48, 48, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2_1 (MaxPooling2D)          │ (None, 24, 24, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop2_1 (Dropout)               │ (None, 24, 24, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool3_1 (MaxPooling2D)          │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop3_1 (Dropout)               │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 4)              │        73,732 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 371,204 (1.42 MB)

 Trainable params: 370,692 (1.41 MB)

 Non-trainable params: 512 (2.00 KB)

In [28]:
def trainModel(model, epochs, optimizer):
    batch_size = 32
    model.compile(optimizer=optimizer, loss = "sparse_categorical_crossentropy", metrics=["accuracy"])
    return model.fit(train_generator, epochs = epochs, batch_size=batch_size)

In [29]:
with tf.device("GPU:0"):
    model_running = trainModel(model=model, epochs=10, optimizer='AdamW')

Epoch 1/10


2024-03-22 00:37:48.706457: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


381/381 ━━━━━━━━━━━━━━━━━━━━ 70s 156ms/step - accuracy: 0.3039 - loss: 3.3393
Epoch 2/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 58s 151ms/step - accuracy: 0.3453 - loss: 1.3748
Epoch 3/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 60s 158ms/step - accuracy: 0.3750 - loss: 1.3302
Epoch 4/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 61s 159ms/step - accuracy: 0.3866 - loss: 1.3106
Epoch 5/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 59s 155ms/step - accuracy: 0.3912 - loss: 1.3118
Epoch 6/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 59s 156ms/step - accuracy: 0.4132 - loss: 1.2917
Epoch 7/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 58s 152ms/step - accuracy: 0.4044 - loss: 1.2930
Epoch 8/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 57s 149ms/step - accuracy: 0.4044 - loss: 1.2871
Epoch 9/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 57s 150ms/step - accuracy: 0.4068 - loss: 1.2845
Epoch 10/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 59s 154ms/step - accuracy: 0.3965 - loss: 1.2935
